<a href="https://colab.research.google.com/github/TaegeunSONG/knu/blob/main/web_image_collecting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 2023.05.25 KNU 코랩버젼 크롤링, 짜집기...진행중


# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.jhjwv6PxJM/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.vjVoXb6sS5/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.jIhIPAW2U5/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [9]:
import os
import sys
import urllib.request
from time import sleep
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import urllib.request 
import shutil

# 검색어    
keyword = '아이유'
# 크롤링할 이미지의 갯수
maxImages = 10

# 프로젝트에 미리 생성해놓은 crawled_img폴더 안에 하위 폴더 생성
# 폴더명에는 입력한 키워드, 이미지 수 정보를 표시함
path = 'crawled_img/'+keyword+'_'+str(maxImages)

try:
    # 중복되는 폴더 명이 없다면 생성
    if not os.path.exists(path):
        os.makedirs(path)
    # 중복된다면 문구 출력 후 프로그램 종료
    else:
        # print('이전에 같은 [검색어, 이미지 수]로 다운로드한 폴더가 존재합니다.')
        # sys.exit(0)
        # os.rmdir(path) # 빈폴더 지울때
        shutil.rmtree(path)
        os.makedirs(path)

except OSError:
    print ('os error')
    sys.exit(0)


#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정, 브라우저 안띄울때, 코랩
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 웹 브라우저 실행
driver = webdriver.Chrome('chromedriver', options=options)

# 검색엔진 설정
driver.get('https://www.google.co.kr/imghp?hl=ko&tab=wi&ogbl')
elem = driver.find_element(By.NAME, 'q') #검색창을 찾아라
elem.send_keys(keyword) # 검색어를 넣어라
elem.send_keys(Keys.RETURN) # 검색해라

last_height = driver.execute_script("return document.body.scrollHeight")
while True: #검색 결과들을 스크롤해서 미리 로딩해둠.
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load page
    time.sleep(2)
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        try:
            driver.find_element(By.CSS_SELECTOR,".mye4qd").click()
        except:
            break
    last_height = new_height



links=[]
images = driver.find_elements(By.CSS_SELECTOR,".rg_i.Q4LuWd")
print( '찾은 이미지는 {n}개 입니다'.format(n=len(images)) )

#찾은 이미지의 원본 링크 추출
count = 0
for image in images:
    if image.get_attribute('src')!=None:
        links.append(image.get_attribute('src'))
    count=count+1
    if count == maxImages:
      break

print(keyword+' 찾은 이미지 개수:',len(links),'개의 링크를 저장하였습니다')
time.sleep(2)

print(path)
for k,i in enumerate(links):
    url = i
    start = time.time()
    time.sleep(1)
    urllib.request.urlretrieve(url, path + '/' + keyword+"_"+str(k)+".jpg")
    print(str(k+1)+'/'+str(len(links))+' '+keyword+' 다운로드 중....... Download time : '+str(time.time() - start)[:5]+' 초')
print(keyword+' ---다운로드 완료---')


driver.close()



찾은 이미지는 635개 입니다
아이유 찾은 이미지 개수: 10 개의 링크를 저장하였습니다
crawled_img/아이유_10
1/10 아이유 다운로드 중....... Download time : 1.001 초
2/10 아이유 다운로드 중....... Download time : 1.000 초
3/10 아이유 다운로드 중....... Download time : 1.000 초
4/10 아이유 다운로드 중....... Download time : 1.001 초
5/10 아이유 다운로드 중....... Download time : 1.001 초
6/10 아이유 다운로드 중....... Download time : 1.001 초
7/10 아이유 다운로드 중....... Download time : 1.001 초
8/10 아이유 다운로드 중....... Download time : 1.000 초
9/10 아이유 다운로드 중....... Download time : 1.002 초
10/10 아이유 다운로드 중....... Download time : 1.001 초
아이유 ---다운로드 완료---
